In [ ]:
# github test

In [1]:
import tweepy
import re
import pandas as pd
import numpy as np
from textblob import TextBlob

In [2]:
consumer_key = "yJI7VztRV1IFd52g5Qm6CoQ4k"   
consumer_secret = "yN2z5rlWziN9Wdi9lB9XjLvogl5ItqXoBlcvck8S2N7mj3FtfF"   
access_token = "909527697646395393-HUEr0Aws7DCnn3VWGrWnMakmPY0SlAI"
access_token_secret = "4ntwDjAKihF9ZKELxMaxRc4GqMd1A9yAamlbmFF7kjCtE"


In [3]:
# Tweepy settings
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)   
auth.set_access_token(access_token, access_token_secret)   
api = tweepy.API(auth)   


In [37]:
def search_tweet(fromdate, todate, keywords, searchsize): # e.g. query='vegan', fromDate='201101012315', toDate='201102012315',).items(100)
    idlist=[]
    tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query=keywords, fromDate=fromdate, toDate=todate,).items(searchsize)
    for tweet in tweets: 
        idlist.append(tweet.id)
    
    
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    created_time=[]

    while len(idlist)>=100:
        idlistcurr=idlist[:100]
        status2=api.statuses_lookup(idlistcurr,tweet_mode="extended")


        for status in status2:
            status_dict = dict(vars(status))
            keys = status_dict.keys()
            single_tweet_data = {"user": status.user.screen_name}
            for k in keys:
                try:
                    v_type = type(status_dict[k])
                except: # make sure no none error
                    v_type = None
                if v_type != None: # make sure no none error
                    if v_type in allowed_types:
                        single_tweet_data[k] = status_dict[k]
                        columns.add(k)
            tweets_data.append(single_tweet_data)
            created_time.append(status.created_at)
        idlist=idlist[100:]


    header_cols = list(columns)
    header_cols.append("user")

    df = pd.DataFrame(tweets_data, columns=header_cols)
    df['created_time']=created_time
    return df


In [43]:
# df.to_csv('sample 600 result.csv')

In [41]:
df.columns

Index(['in_reply_to_user_id_str', 'full_text', 'id', 'in_reply_to_screen_name',
       'in_reply_to_user_id', 'source_url', 'in_reply_to_status_id_str',
       'favorite_count', 'lang', 'source', 'retweet_count', 'id_str',
       'in_reply_to_status_id', 'user', 'created_time'],
      dtype='object')

In [36]:
# search 20 item and store them in df
# 'vegan', English(no), 2011 Jan-Feb
# handling retweets, show full text

# for Sandbox user, limits are 50 requests and 5k tweets per month. 
# there's no limit for the each search though.the last test for the searchsize is 600 and it works. 

idlist=[]
tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query='vegan', fromDate='201101012315', toDate='201102012315',).items(100)
for tweet in tweets: 
    idlist.append(tweet.id)

# if searchsize <=100:     
# status2=api.statuses_lookup(idlist,tweet_mode="extended")
# for status in status2:
#     try:
#         print(status.retweeted_status.full_text)
#     except AttributeError:  # Not a Retweet
#         print(status.full_text)

# if searchsize >100:

columns = set()
allowed_types = [str, int]
tweets_data = []
created_time=[]
    
while len(idlist)>=100:
    idlistcurr=idlist[:100]
    status2=api.statuses_lookup(idlistcurr,tweet_mode="extended")
    

    for status in status2:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"user": status.user.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except: # make sure no none error
                v_type = None
            if v_type != None: # make sure no none error
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)
        created_time.append(status.created_at)
    idlist=idlist[100:]

        
header_cols = list(columns)
header_cols.append("user")
# header_cols.append('created_time')

df = pd.DataFrame(tweets_data, columns=header_cols)
df['created_time']=created_time


TweepError: {'message': 'Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.', 'sent': '2021-04-26T10:52:53+00:00', 'transactionId': '00a174f100a23dc7'}

In [29]:
df = pd.DataFrame(tweets_data, columns=header_cols)
df['created_time']=created_time
df.head()


,in_reply_to_user_id_str,full_text,id,in_reply_to_screen_name,in_reply_to_user_id,source_url,in_reply_to_status_id_str,favorite_count,lang,source,retweet_count,id_str,in_reply_to_status_id,user,created_time
0,14205819,Are you a #vegan mom? Have some advice for nu...,32576188593475585,dopemamaJ,14205819.0,http://twitter.com,32558219918311424,0,en,Twitter Web Client,0,32576188593475585,3.255822e+16,vegansalt,2011-02-01 23:09:03
1,19468868,@Shep133 I had a vegan burger the other day wi...,32576131479633920,Shep133,19468868.0,http://twitter.com/download/android,32572192180936704,0,en,Twitter for Android,0,32576131479633920,3.257219e+16,leja99,2011-02-01 23:08:49
2,NaN,Overall GRATEFUL @TheOprahShow did a vegan sho...,32577369881452544,NaN,NaN,http://www.ubertwitter.com/bb/download.php,NaN,0,en,ÜberSocialOrig,4,32577369881452544,NaN,simonereyes,2011-02-01 23:13:44
3,114023803,@DreddedNupe @sableverity vegan mac n cheese i...,32577076951261184,RalphInCharge,114023803.0,http://www.echofon.com/,32575282254708737,0,en,Echofon,0,32577076951261184,3.257528e+16,SusannaDW,2011-02-01 23:12:35
4,NaN,RT @Chichi_C: Whyyyy?? **RT @naijafabulosity: ...,32576332147720192,NaN,NaN,http://www.ubertwitter.com/bb/download.php,NaN,0,en,ÜberSocialOrig,0,32576332147720192,NaN,Zeetherapy,2011-02-01 23:09:37


In [19]:
df.loc[4].full_text

"RT @Chichi_C: Whyyyy?? **RT @naijafabulosity: Making my vegan shopping list. &gt;Lol I cld say I'm trying to be healthier but to lose 10 pounds"

In [43]:
# store all features
# ref: from 26min, https://www.youtube.com/watch?v=dvAurfBB6Jk&t=2637s&ab_channel=CodingEntrepreneurs 
# need further editing. what's difference between author and user?


search=api.search(q='vegan', lang='en', count=5,tweet_mode='extended')

columns = set()
allowed_types = [str, int]
tweets_data = []
for status in search:
    status_dict = dict(vars(status))
    keys = status_dict.keys()
    single_tweet_data = {"user": status.user.screen_name}
    for k in keys:
        try:
            v_type = type(status_dict[k])
        except: # make sure no none error
            v_type = None
        if v_type != None: # make sure no none error
            if v_type in allowed_types:
                single_tweet_data[k] = status_dict[k]
                columns.add(k)
    tweets_data.append(single_tweet_data)

        
header_cols = list(columns)
header_cols.append("user")

df = pd.DataFrame(tweets_data, columns=header_cols)
df

500

In [16]:
# try full archive

tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query='vegan', fromDate='201101012315', toDate='201201012315').items(20)
for tweet in tweets:     
    print(tweet.text, tweet.created_at) 

$1 to the Ethiopian Homeless Feeding Program (vegan) for each e-mail sign-up in January (up to $3,000) --&gt; DAY 1... http://t.co/ICduihnY 2012-01-01 23:14:57
@enterblisstonia oooh. tofu seitan scramble from vegan yum yum (make your own seitan if you have a food processor)... 2012-01-01 23:14:50
“@lizabitty: Making vegan "beef" stew. Good way to start the year. Yummy smell throughout the house!” 2012-01-01 23:14:46
FGV flashback: vegan steak potato crisps/chips http://t.co/Y0mcyuqw 2012-01-01 23:14:36
Best raw, vegan, and organic foods http://marty_642.tripod.com/bestrawveganandorganicfoods.html 2012-01-01 23:14:33
Vegan for reviews http://t.co/8K9xSLeB 2012-01-01 23:14:24
Making vegan "beef" stew. Good way to start the year. Yummy smell throughout the house! 2012-01-01 23:14:14
RT @OneGreenPlanet: Happy New Year One Green Planeteers! Looking forward to spreading more good ideas for people, animals and the planet ... 2012-01-01 23:14:12
having some awesome @Engine2Diet #vegan gnocchi 

In [27]:
# try get tweet id from full archive, and use id to get full text 

idlist=[]
tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query='vegan', fromDate='202001012315', toDate='202001022315').items(20)
for tweet in tweets:     
    idlist.append(tweet.id)
    


In [16]:
# try full archive

tweets=tweepy.Cursor(api.search_full_archive,
                     environment_name='Development', query='vegan', fromDate='201101012315', toDate='201201012315').items(20)
for tweet in tweets:     
    print(tweet.text, tweet.created_at) 

$1 to the Ethiopian Homeless Feeding Program (vegan) for each e-mail sign-up in January (up to $3,000) --&gt; DAY 1... http://t.co/ICduihnY 2012-01-01 23:14:57
@enterblisstonia oooh. tofu seitan scramble from vegan yum yum (make your own seitan if you have a food processor)... 2012-01-01 23:14:50
“@lizabitty: Making vegan "beef" stew. Good way to start the year. Yummy smell throughout the house!” 2012-01-01 23:14:46
FGV flashback: vegan steak potato crisps/chips http://t.co/Y0mcyuqw 2012-01-01 23:14:36
Best raw, vegan, and organic foods http://marty_642.tripod.com/bestrawveganandorganicfoods.html 2012-01-01 23:14:33
Vegan for reviews http://t.co/8K9xSLeB 2012-01-01 23:14:24
Making vegan "beef" stew. Good way to start the year. Yummy smell throughout the house! 2012-01-01 23:14:14
RT @OneGreenPlanet: Happy New Year One Green Planeteers! Looking forward to spreading more good ideas for people, animals and the planet ... 2012-01-01 23:14:12
having some awesome @Engine2Diet #vegan gnocchi 

In [28]:
idlist

[1212874962144509953,
 1212874956620664832,
 1212874954972246017,
 1212874946801815552,
 1212874939247845377,
 1212874921723998208,
 1212874910739116032,
 1212874901998178304,
 1212874895497019394,
 1212874890874884096,
 1212874883123728385,
 1212874876295532544,
 1212874875620249601,
 1212874872122171392,
 1212874870733901825,
 1212874841772220423,
 1212874840216084480,
 1212874834096599048,
 1212874829596184577,
 1212874828572704769]

In [35]:
# this works for search result of year 2020 but not for 2011
# is full text only available for recent years?

status = api.get_status(1212874890874884096, tweet_mode="extended")
try:
    print(status.retweeted_status.full_text)
except AttributeError:  # Not a Retweet
    print(status.full_text)

Happy New Year 2020! For a #healthy start, why not try TIANA 100% Pure MCT Energy, a natural, #vegan, pure and powerful supplement to beat tiredness and boost metabolism, easy to mix in your #morning #coffee, #tea, #smoothies and much more. #energy #bodyenergy #veganuary #mctoil https://t.co/IelmqqYvHW


In [37]:
# search a list of tweet ids
status2=api.statuses_lookup(idlist,tweet_mode="extended")
for status in status2:
    try:
        print(status.retweeted_status.full_text)
    except AttributeError:  # Not a Retweet
        print(status.full_text)


Imagine if McDonald’s did a vegan Big Mac or a vegan chicken selects/legend meal. They’d make a killing. A huge market for them waiting and here they are releasing a different shape of the mushy veg patty once again. So bizarre
Islamophobes losing their shit because Warburtons has placed the Halal certification logo on their packaging.

It's vegan &amp; kosher too. ie: no meat,fat or dairy. ANYTHING vegan/vegetarian is Halal.

Milk &amp; fish are too

Do they not eat if they holiday in Turkey, Tunisia or Egypt?
Is your #NewYearsResolution to eat #healthy? Sign up for our Green Plate Club! Details:
https://t.co/B9uCLoeXNn

GOOD CHOICE KITCHEN
#organic #vegan #glutenfree
147 Main St #Ossining
#Westchester #NewYork
914-930-1591
#wholefood #plantbased #wfpb https://t.co/usDrFnP8O5
The Golden Globes Will Serve a 100% Vegan Menu https://t.co/Zh2pkKcaTc
Decided to try the BK Impossible Whopper. 

It was very good and I prob prefer it over the original because it’s not greasy but still has all

In [15]:
# basic timeline for Anran's account
public_tweets = api.home_timeline()
for tweet in public_tweets:     
    print(tweet.text, tweet.created_at) 

RT @L_Brierley: 🎓Interested in a #mastersdegree #MSC in #stats and #datascience for #health?

Next week is Postgrad Open Week @LivUni @livu… 2021-04-08 13:24:59
How to behave in a commercial gallery, if you’ve never dared set foot in one https://t.co/bnJMq2OkkB 2021-04-08 13:15:00
Do you work for an England-based #charity that's undergone a successful cultural change in the way your organisatio… https://t.co/BH7PK5dZxa 2021-04-08 13:10:02
✨ Shoot your shot.

📷 ‘Redefining Ballerism: Upping the Ante’, 2009 by Seattle-based metal artist #JanaBrevick… https://t.co/LQ14hBkpu8 2021-04-08 13:07:31
RT @MLBrook: and its not like we lack for great orgs working on data at the local level either... cc @opendatamcr @ODILeeds 2021-04-08 12:45:53
The biannual “Taking Stock of Teens,” report, which surveys over 7,000 teens showed what brands and retailers can e… https://t.co/HkxRN7OU4V 2021-04-08 12:34:00
The artist Frederic William Burton, who was born in County Wicklow #OTD in 1816, may be little k

In [16]:
# Get the User object 
user = api.get_user('AnranZhao')
print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
   print(friend.screen_name)

AnranZhao
17
UoM_ITS
BeveragesHot
SorchaUKDS
GillUKDS
CMI_Man
Patty_NZ
VanessaAHiggins
ABloom_20
UKDSImpact
mikybartoli
JKasmireComplex
_JAStark
PyDataMCR
JosephAllen1234
opendatamcr
UKDataService
stephen_whitemn
McrContemporary
OfficialUoM
arthistsoc_uom


In [7]:
# collect 1000 English tweets on 'vegan' and show the full text
# but all retweets are not full 
# geolocation restriction is added
for status in tweepy.Cursor(api.search, q='vegan', lang='en', tweet_mode='extended', geocode="31.578871,74.305184,5km" ).items(1000):
    print (status.full_text)

RT @BaymaxTrust: Friends not food 🐷🐮🐔
Feel free to share 🙌

👉(via: @theanarchistavocado)
✔Follow @BaymaxTrust  for more 🌱

#vegan #bhent #I…
RT @BaymaxTrust: Friends not food 🐷🐮🐔
Feel free to share 🙌

👉(via: @theanarchistavocado)
✔Follow @BaymaxTrust  for more 🌱

#vegan #bhent #I…
RT @BaymaxTrust: Friends not food 🐷🐮🐔
Feel free to share 🙌

👉(via: @theanarchistavocado)
✔Follow @BaymaxTrust  for more 🌱

#vegan #bhent #I…
RT @BaymaxTrust: Friends not food 🐷🐮🐔
Feel free to share 🙌

👉(via: @theanarchistavocado)
✔Follow @BaymaxTrust  for more 🌱

#vegan #bhent #I…
RT @BaymaxTrust: Friends not food 🐷🐮🐔
Feel free to share 🙌

👉(via: @theanarchistavocado)
✔Follow @BaymaxTrust  for more 🌱

#vegan #bhent #I…
Friends not food 🐷🐮🐔
Feel free to share 🙌

👉(via: @theanarchistavocado)
✔Follow @BaymaxTrust  for more 🌱

#vegan #bhent #India #health #Heart #RightToFood #GoVegan #LifeGoesOn #animals #consciousliving #COVID19 #consciousness #climate #climatechange #savetheplanet #diet https://t.co/SGPXjYXqeU


RT @KalraVimi: Everyone is an animal lover until you mention the ones on their plate .  Don't be a hypocrite,  Go vegan 🌱
#endspeciesism 
#…
Vegan world will be a compassionate world .Come lets create one 💚❤.
Learn to respect the fellow earthlings.
Unlearn the casual blindness towards the sufferings of the innocent animals.
Relearn the forgotton compassion.
#GoVegan
#endspeciesism
Everyone is an animal lover until you mention the ones on their plate .  Don't be a hypocrite,  Go vegan 🌱
#endspeciesism 
#GoVegan
RT @AmjorDissha: Happy World Vegan Day To All the Vegans 
And a short message for All Nonvegans 😊 watch it🐣 
#1stNovember #worldveganday #b…
We need #Leaders not Politicians.

#youth
#noneedmorespeech
#nomorelie
#bhagatsinghsherepunjab
#inquilabzindabad
#businessgoals
#vegan
#NoFarmersNoFood
#FarmersProtests
#PMCareScam
#India
#Worldwide
#hangrycub
#NoVoteToBJP
#qualifiedLeader
#JaiBhim
#KharchaPeCharcha
RT @NPDay: Modi Govt cited the case of a BBC correspondent who filed 240 RTI

In [18]:
# collect 200 tweets from user TheVeganRD and show the full text
# NB, here retweets lost the identification 'RT' and become common tweets

results = api.user_timeline(id='TheVeganRD', count=200, tweet_mode='extended')
for status in results:
    try:
        print(status.retweeted_status.full_text)
    except AttributeError:  # Not a Retweet
        print(status.full_text)

@Mjwenckus Some combination of salt, fat, flavor, texture, umami. After all, the early nondairy cheeses contained casein and they were anything but addictive!
I've been writing vegan advocacy material since the early 1990s. I'm totally disgusted by the lack of credibility in some of the most oft-repeated claims, just as I know my friend @TheVeganRD has equal despair over bogus vegan health claims. 5/
If someone says you can lose weight without being on a "diet" by eating plant-based and then they say you can never have oil or sugar or veggie meats or refined pasta--guess what? They’re putting you on a diet.
@SuchFun_AreWe Agree -- I am a very big fan of Violife! It's on sale right now and my refrigerator is packed with it.
@mdryja I just looked it up -- and now it's on my list for the next time I'm in southern CA.
@tempehlover @moozoogabba @SuchFun_AreWe That cheese looks amazing!
@NCanimalvoters @jan_dutkiewicz Oh, those corn dogs!
Such an interesting article on the history and evolut

In [19]:
# comparison
results = api.user_timeline(id='TheVeganRD', count=200, tweet_mode='extended')
for status in results:
    print(status.full_text)

@Mjwenckus Some combination of salt, fat, flavor, texture, umami. After all, the early nondairy cheeses contained casein and they were anything but addictive!
RT @vegan: I've been writing vegan advocacy material since the early 1990s. I'm totally disgusted by the lack of credibility in some of the…
If someone says you can lose weight without being on a "diet" by eating plant-based and then they say you can never have oil or sugar or veggie meats or refined pasta--guess what? They’re putting you on a diet.
@SuchFun_AreWe Agree -- I am a very big fan of Violife! It's on sale right now and my refrigerator is packed with it.
@mdryja I just looked it up -- and now it's on my list for the next time I'm in southern CA.
@tempehlover @moozoogabba @SuchFun_AreWe That cheese looks amazing!
@NCanimalvoters @jan_dutkiewicz Oh, those corn dogs!
Such an interesting article on the history and evolution of #vegan cheese. (Just skip over the questionable claims about "addictive" properties of casein.)  

In [22]:
# test: without cursor, only tweets on the first result page is shown, which is 100 tweets here
count=0
res=api.search(q='veganism', lang='en',tweet_mode='extended', count=100000)
for tweet in res:
    count+=1
    print(tweet.full_text)
    print(count)

Veganism is a social justice movement. The way we fight for slavery, racism, women rights... now is time to fight for animals. #GoVegan #endspeciesism
1
Veganism &amp; spirituality do not go hand and hand. Argue with yo mama while I eat my cheese &amp; chicken nuggies. https://t.co/BDM4W4y7tF
2
@jewish_activist the way most leftists talk about animal rights is on par with how ben shapiro “debates”. for all the complaints about “white veganism” they sure love using BIPOC people as shields for their own shitty habits
3
RT @i_am_RAGHAV: There is no argument against veganism. Stop being a plant rights activist. Stop the slaughter, exploitation, enslavement,…
4
@makeyallsweat @Klanderverdig @milkthecashews Firstly, I have clarified several times, vegans have contracted the virus. Secondly, I stand by my choice and the right to choose without being judged. Thirdly, you say veganism can't fix it, when the science says Covid-19 jumped from animals to humans. Same as HIV, SARS etc.
5
RT @Veggie

In [33]:
# store only text and time

df=pd.DataFrame()
content=[]
time=[]

search=api.search(q='COVID', lang='en', count=5,tweet_mode='extended')
for tweet in search:
    content.append(tweet.full_text)
    time.append(tweet.created_at )
    
df['content']=content
df['time']=time
df

,content,time
0,RT @PChidambaram_IN: The Centre is targeting M...,2021-04-08 13:55:22
1,RT @ShapedInternet: It's been a year since we ...,2021-04-08 13:55:22
2,RT @ArshiyaMuskaan7: Respected @DrTamilisaiGuv...,2021-04-08 13:55:22
3,RT @WebMD: The Pfizer and Moderna COVID-19 vac...,2021-04-08 13:55:21
4,RT @PoliticsForAlI: I would be more likely to ...,2021-04-08 13:55:21


In [29]:
# store all features
# ref: from 26min, https://www.youtube.com/watch?v=dvAurfBB6Jk&t=2637s&ab_channel=CodingEntrepreneurs 
# need further editing. what's difference between author and user?


search=api.search(q='vegan', lang='en', count=5,tweet_mode='extended')

columns = set()
allowed_types = [str, int]
tweets_data = []
for status in search:
    status_dict = dict(vars(status))
    keys = status_dict.keys()
    single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
    for k in keys:
        try:
            v_type = type(status_dict[k])
        except:
            v_type = None
        if v_type != None:
            if v_type in allowed_types:
                single_tweet_data[k] = status_dict[k]
                columns.add(k)
    tweets_data.append(single_tweet_data)

        
header_cols = list(columns)
header_cols.append("user")
header_cols.append('author')

df = pd.DataFrame(tweets_data, columns=header_cols)
df

,in_reply_to_user_id_str,retweet_count,id,in_reply_to_status_id_str,in_reply_to_screen_name,id_str,in_reply_to_user_id,source,full_text,favorite_count,in_reply_to_status_id,lang,source_url,user,author
0,NaN,6,1380156533716226056,NaN,NaN,1380156533716226056,NaN,Twitter for iPhone,RT @Unproblemattic: An Anti-Vaxxer and a Vegan...,0,NaN,en,http://twitter.com/download/iphone,CallmeQueen5,CallmeQueen5
1,NaN,0,1380156520671948800,NaN,NaN,1380156520671948800,NaN,Twitter Web App,vegan greek yoghurt is so good,0,NaN,en,https://mobile.twitter.com,doubledepress0,doubledepress0
2,NaN,58,1380156508994998275,NaN,NaN,1380156508994998275,NaN,Twitter for Android,RT @KetoAurelius: To be at optimal health on a...,0,NaN,en,http://twitter.com/download/android,CopyOfOne,CopyOfOne
3,NaN,0,1380156493543133191,NaN,NaN,1380156493543133191,NaN,IFTTT,Beyond Cabbage: How to Spice Up Your Sauerkrau...,0,NaN,en,https://ifttt.com,PlantBasedGent,PlantBasedGent
4,327187863,0,1380156476367515658,1380149382671167496,Wxcafe,1380156476367515658,327187863.0,Twitter Web App,@Wxcafe it’s not too hard to forge a vegan ID.,0,1.380149e+18,en,https://mobile.twitter.com,Q_quiscula,Q_quiscula


In [33]:
df.loc[0].content

'RT @MSNBC: WATCH: President Biden urges Americans to continue following Covid-19 guidelines: "Mask up, it’s a patriotic duty. It’s the only…'